In [ ]:
from IPython.display import Audio
from datasets import load_dataset, load_from_disk
import os
import gc
import shutil
import torch
from scipy.io import wavfile
import numpy as np
from pydub import AudioSegment
from scipy import signal
import kagglehub

In [16]:
# Download latest version
path = kagglehub.dataset_download("deeptanshumalu/mathbridge-audio-files")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\deept\.cache\kagglehub\datasets\deeptanshumalu\mathbridge-audio-files\versions\1


In [42]:
ds = load_dataset("speech")
ds

Resolving data files:   0%|          | 0/99986 [00:00<?, ?it/s]

Computing checksums:  84%|########4 | 84276/99986 [00:05<00:00, 16851.73it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 99986
    })
})

In [44]:
ds["train"].save_to_disk("mathbridge_audio_arrays")

Saving the dataset (0/33 shards):   0%|          | 0/99986 [00:00<?, ? examples/s]

In [50]:
ds_array = load_from_disk("mathbridge_audio_arrays")
ds_array

Loading dataset from disk:   0%|          | 0/33 [00:00<?, ?it/s]

Dataset({
    features: ['audio'],
    num_rows: 99986
})

In [54]:
ds_array[34324]

{'audio': {'path': 'tts_40900.mp3',
  'array': array([-0.00271606, -0.00820923, -0.00604248, ...,  0.        ,
          0.        ,  0.        ]),
  'sampling_rate': 22050}}

In [ ]:
# source_dir = "/kaggle/input/mathbridge-filtered"

# # Destination directory
# destination_dir = "/kaggle/working/mathbridge-filtered"

# shutil.copytree(source_dir, destination_dir)

In [2]:
ds_train = load_from_disk("mathbridge_filtered")
ds_train

Dataset({
    features: ['context_before', 'equation', 'context_after', 'spoken_English'],
    num_rows: 4753354
})

In [48]:
ds_train[0]

{'context_before': 'are modeled by a stochastic control process with variance',
 'equation': '$ \\sigma^2_t $',
 'context_after': 'controlled by the agent and with a mean of zero . This models potential effect of actions centered around the null action . To compute various quantities of interest ,',
 'spoken_English': 'sigma squared sub t.'}

In [74]:
# Define a function to map the text to a file path
def map_to_speech_file(example, idx):
    if idx%100==0:
        print(idx)
    path = example["audio"]["path"]
    file_no = int(path.split("_")[-1].split(".")[0])
    audio_array = example["audio"]["array"]
    example["equation"] = ds_train[file_no]["equation"]
    example["spoken_English"] = ds_train[file_no]["spoken_English"]
    num_samples = int(len(audio_array) * 16000 / 22050)
    new_audio_array = signal.resample(audio_array, num_samples)
    example["audio"]["sampling_rate"] = 16000
    example["audio"]["array"] = new_audio_array
    example["audio"]["path"] = path
    del audio_array, num_samples, new_audio_array, file_no, path
    gc.collect()
    return example

# Apply the function to the dataset
ds_speech = ds_array.select(range(100)).map(map_to_speech_file, with_indices=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

0


In [76]:
ds_speech[43]

{'audio': {'path': None,
  'array': array([-0.05181885,  0.0401001 , -0.00469971, ...,  0.01113892,
         -0.01516724,  0.02261353]),
  'sampling_rate': 16000},
 'equation': '$ T_0 > 0 $',
 'spoken_English': 'T sub 0 is greater than zero.'}

In [ ]:
ds_speech.save_to_disk("/kaggle/working/mathbridge_speech")

In [ ]:
shutil.rmtree("/kaggle/working/mathbridge-filtered")
# os.remove("/kaggle/working/state.db")